## **双低可转债轮动**
* 参考集思录帖子https://www.jisilu.cn/question/273614
* 对可转债价格和溢价率*100进行相加，值越小排名越排前
* 选取样本排名靠前的10只，每月轮动
----

In [8]:
import requests
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import json
import datetime as dt
import math

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline
# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [9]:
class ConvertBond():
    """
    类：从集思路网站爬取可转债相关数据
    函数说明：
    getBondData:       爬取，清洗数据，返回DataFrame
    getBondFactor:     按照参数设定的columns，获取相应的数据，返回DataFrame
    getBondColumns:    获取columns，具体含义须对照集思路，返回list
    getStockSingleVIX: 获取单个股票最近一年的年化波动率，基于tushare，返回float
    getStockVIX:       获取所有可转债对应正股的年化波动率，返回DataFrame
    """
    def getBondData(self):
        url = 'https://www.jisilu.cn/data/cbnew/cb_list/'
        html = requests.get(url)
        data = html.json()
        data = data['rows']

        data_per = DataFrame()
        data_per_temp = DataFrame()
        for i in range(len(data)):
            data_per_temp = DataFrame(DataFrame(data[i])['cell'])
            data_per = pd.concat([data_per, data_per_temp], axis=1)
        bond_data = data_per.T
        bond_data.index = np.arange(1, len(bond_data) + 1)
        return bond_data

    def getBondFactor(self, *args):
        data = DataFrame()
        data_temp = DataFrame()
        for col in args:
            if col in self.getBondData().columns.tolist():
                data_temp = self.getBondData()[col]
                data = pd.concat([data, data_temp], axis=1)
            else:
                raise KeyError('Wrong column! Key: %s' % col)
        return data

    def getBondColumns(self):
        res = self.getBondData().columns.tolist()
        # print(res)
        return res

    def getStockSingleVIX(self, code):
        end = datetime.now().strftime('%Y-%m-%d')
        start = (datetime.now() - timedelta(days=252)).strftime('%Y-%m-%d')
        stock_data = ts.get_k_data(code, start=start, end=end)
        stock_change_per_day = stock_data['close'].pct_change()
        VIX = stock_change_per_day.dropna().std() * math.sqrt(252)
        return VIX

    def getStockVIX(self):
        data = self.getBondFactor('stock_id', 'stock_nm')
        data['VIX'] = data['stock_id'].apply(lambda x: x[2:]).apply(lambda x: self.getStockSingleVIX(x))
        data['VIX'] = np.round(data['VIX'] * 100, 2)
        # print(data)
        return data

In [10]:
df_bond = ConvertBond().getBondData()

In [11]:
df_bond.to_excel('./test.xlsx')

In [12]:
len(df_bond)

172